<a href="https://colab.research.google.com/github/shourya03roy/Attendance-prediction/blob/main/Attendance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [ ]:
  # # Setting random seed for reproducibility
  # np.random.seed(42)

  # # Generate synthetic data for the dataset
  # data = {
  #     'Past Attendance Records (%)': np.random.randint(60, 100, size=1000),
  #     'Course Schedule': np.random.choice(['Morning', 'Afternoon', 'Evening'], size=1000),
  #     'Weather Conditions': np.random.choice(['Clear', 'Rainy'], size=1000),
  #     'Day of the Week': np.random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], size=1000),
  #     'Extracurricular Activities': np.random.choice(['None', 'Sports', 'Music', 'Part-time Job'], size=1000),
  #     'Academic Performance': np.random.choice(['Poor', 'Average', 'Good'], size=1000),
  #     'Proximity to Exams or Assignments': np.random.choice(['Low', 'Medium', 'High'], size=1000),
  #     'Health Status': np.random.choice(['Healthy', 'Unwell'], size=1000),
  #     'Absent': np.random.randint(0, 2, size=1000)  # Simulating absence (1) or not (0)
  # }
  np.random.seed(42)

  # Generate past attendance records based on academic performance and proximity to exams
  academic_performance = np.random.choice(['Poor', 'Average', 'Good'], size=1000)
  proximity_to_exams = np.random.choice(['Low', 'Medium', 'High'], size=1000)
  past_attendance_records = np.where( (academic_performance == 'Good'),
                                    np.concatenate(( np.random.randint(80, 100, size=900),np.random.randint(40, 60, size=100))),
                                    np.concatenate((np.random.randint(60, 80, size=250),np.random.randint(20, 40, size=250),np.random.randint(40, 60, size=500))))

  # Generate course schedule based on academic performance
  course_schedule = np.where(academic_performance == 'Good',
                            np.concatenate(( np.random.choice(['Morning', 'Afternoon'], size=900), np.random.choice(['Afternoon'], size=100) )),
                            np.random.choice(['Afternoon', 'Evening'], size=1000))


  # Generate weather conditions based on day of the week
  day_of_the_week = np.random.choice(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'], size=1000)
  weather_conditions = np.random.choice(['Sunny','Cloudy', 'Rainy'], size=1000)

  # Generate extracurricular activities based on academic performance
  extracurricular_activities = np.where(academic_performance == 'Good',
                                        np.random.choice(['None', 'Sports'], size=1000),
                                        np.random.choice(['None', 'Music', 'Part-time Job'], size=1000))

  # Generate health status based on extracurricular activities
  health_status = np.where(extracurricular_activities == 'Sports',
                          np.random.choice(['Healthy', 'Unwell'], size=1000, p=[0.8, 0.2]),
                          np.random.choice(['Healthy', 'Unwell'], size=1000))

  # Simulate absence based on past attendance records, course schedule, and weather conditions
  absent = np.where((past_attendance_records < 50) | ((course_schedule == 'Evening') & (weather_conditions == 'Rainy') & (health_status == 'Unwell')),
                    np.random.choice([1, 0], size=1000, p=[0.8, 0.2]),
                    np.random.choice([0, 1], size=1000, p=[0.8, 0.2]))

  # Create DataFrame
  data = {
      'Past Attendance Records (%)': past_attendance_records,
      'Course Schedule': course_schedule,
      'Weather Conditions': weather_conditions,
      'Day of the Week': day_of_the_week,
      'Extracurricular Activities': extracurricular_activities,
      'Academic Performance': academic_performance,
      'Proximity to Exams or Assignments': proximity_to_exams,
      'Health Status': health_status,
      'Absent': absent
  }
  # Convert data into a DataFrame
  df = pd.DataFrame(data)

  # Display the first few rows of the dataset
  print(df.head(),"\n\n")

   Past Attendance Records (%) Course Schedule Weather Conditions  \
0                           93       Afternoon             Cloudy   
1                           75         Evening              Sunny   
2                           92       Afternoon              Sunny   
3                           87         Morning              Sunny   
4                           72         Evening             Cloudy   

  Day of the Week Extracurricular Activities Academic Performance  \
0         Tuesday                       None                 Good   
1       Wednesday              Part-time Job                 Poor   
2       Wednesday                     Sports                 Good   
3          Monday                       None                 Good   
4          Friday                      Music                 Poor   

  Proximity to Exams or Assignments Health Status  Absent  
0                              High        Unwell       0  
1                              High       Healthy 

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/Attendance.csv', mode='w', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Attendance.csv')

In [ ]:
# print(df)
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
print(X[:5],"\n\n",y[:5])

   Past Attendance Records (%) Course Schedule Weather Conditions  \
0                           93       Afternoon             Cloudy   
1                           75         Evening              Sunny   
2                           92       Afternoon              Sunny   
3                           87         Morning              Sunny   
4                           72         Evening             Cloudy   

  Day of the Week Extracurricular Activities Academic Performance  \
0         Tuesday                       None                 Good   
1       Wednesday              Part-time Job                 Poor   
2       Wednesday                     Sports                 Good   
3          Monday                       None                 Good   
4          Friday                      Music                 Poor   

  Proximity to Exams or Assignments Health Status  
0                              High        Unwell  
1                              High       Healthy  
2             

In [ ]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print(object_cols)

['Course Schedule', 'Weather Conditions', 'Day of the Week', 'Extracurricular Activities', 'Academic Performance', 'Proximity to Exams or Assignments', 'Health Status']


In [ ]:
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(X[object_cols])

In [ ]:
print(X)
type(X)

     Past Attendance Records (%)  Course Schedule  Weather Conditions  \
0                             93              0.0                 0.0   
1                             75              1.0                 2.0   
2                             92              0.0                 2.0   
3                             87              2.0                 2.0   
4                             72              1.0                 0.0   
..                           ...              ...                 ...   
995                           53              0.0                 0.0   
996                           55              1.0                 0.0   
997                           48              0.0                 1.0   
998                           51              0.0                 2.0   
999                           46              0.0                 2.0   

     Day of the Week  Extracurricular Activities  Academic Performance  \
0                3.0                         1.0 

pandas.core.frame.DataFrame

In [ ]:
# print(X.head
scaler = StandardScaler()
X = scaler.fit_transform(X)
# normalize(X, norm="l2")
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.40,random_state=10)

In [ ]:
model = LogisticRegression(C=0.1,solver="newton-cholesky",penalty="none").fit(X_train,y_train)
model.score(X_test,y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


0.765

In [ ]:
model.score(X_train,y_train)

0.7616666666666667

In [ ]:
X_options = {
    'Past Attendance Records (%)': np.array([0,100]),
    'Course Schedule': np.array(['Morning','Afternoon', 'Evening']),
    'Weather Conditions': np.array(['Sunny','Cloudy', 'Rainy']),
    'Day of the Week': np.array(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']),
    'Extracurricular Activities': np.array(['None', 'Music', 'Part-time Job','Sports']),
    'Academic Performance': np.array(['Poor', 'Average', 'Good']),
    'Proximity to Exams or Assignments': np.array(['Low', 'Medium', 'High']),
    'Health Status': np.array(['Healthy', 'Unwell']),
}

In [ ]:
X_user = np.array([])
for key,range in X_options.items():
    print(key,"has the range ", range)
    ans = input("Enter choice: ")
    X_user = np.append(X_user,ans)
print(X_user)
# X_user = scaler.transform(X[0])
# print(model.predict(X_user.reshape(1,-1)))


Past Attendance Records (%) has the range  [  0 100]
Enter choice: 98
Course Schedule has the range  ['Morning' 'Afternoon' 'Evening']
Enter choice: Morning
Weather Conditions has the range  ['Sunny' 'Cloudy' 'Rainy']
Enter choice: Rainy
Day of the Week has the range  ['Monday' 'Tuesday' 'Wednesday' 'Thursday' 'Friday']
Enter choice: Tuesday
Extracurricular Activities has the range  ['None' 'Music' 'Part-time Job' 'Sports']
Enter choice: Sports
Academic Performance has the range  ['Poor' 'Average' 'Good']
Enter choice: Good
Proximity to Exams or Assignments has the range  ['Low' 'Medium' 'High']
Enter choice: Low
Health Status has the range  ['Healthy' 'Unwell']
Enter choice: Healthy
['98' 'Morning' 'Rainy' 'Tuesday' 'Sports' 'Good' 'Low' 'Healthy']


In [ ]:
X_user_selected = X_user[1:8]
X_user_transformed = ordinal_encoder.transform(X_user_selected.reshape(1,-1))
X_user[1:8] = X_user_transformed
X_user_reshaped = X_user
X_user = scaler.transform(X_user_reshaped.reshape(1,-2))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print(model.predict(X_user.reshape(1,-1)))

[0]


In [ ]:
# Assuming model is your trained logistic regression model
# X_test is your test data

# Get the probability estimates for class 1 (positive class)
y_pred_proba = model.predict_proba(X_train)[:, 1]
y_tpred_proba = model.predict_proba(X_test)[:, 1]

# Apply threshold to convert probabilities into binary predictions
threshold = 0.6  # Change this threshold as needed
y_tpred = (y_tpred_proba >= threshold).astype(int)
y_pred = (y_pred_proba >= threshold).astype(int)
print("Binary predictions with threshold {}: {}\n{}".format(threshold, y_pred,y_tpred))


Binary predictions with threshold 0.6: [0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 0 0 0
 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 0
 1 1 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 0 1 0 0
 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 1 0 1 1 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 1 0 1
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 1
 0 1 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 1 1 1
 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0
 0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 

In [ ]:
from sklearn.metrics import accuracy_score

# Assuming y_true contains the true labels and y_pred contains the predicted labels
accuracytest = accuracy_score(y_test, y_tpred)
accuracy = accuracy_score(y_train, y_pred)
print("training accuracy:", accuracy)
print("testing accuracy:", accuracytest)

training accuracy: 0.7516666666666667
testing accuracy: 0.77


In [ ]:
df.describe()

,Past Attendance Records (%),Absent
count,1000.000000,1000.000000
mean,60.552000,0.436000
std,22.485437,0.496135
min,20.000000,0.000000
25%,43.000000,0.000000
50%,56.000000,0.000000
75%,82.000000,1.000000
max,99.000000,1.000000
